In [ ]:
import numpy as np
from scipy.optimize import root
from equations import *

# Function to solve for Ts at each time step
def solve_Ts(SWin, SWout, LWin, Tair, RH_air, u, prec, snow_cover):
    
    # Define the equation to solve
    def equation_system(Ts):
        
        LWout = LWout(Ts)
        SH = SH(u, Ts, Tair)
        RH_s = surface_relative_humidity(RH_air, Ts, Ta, prec, snow_cover, u)
        q_s = specific_humidity(Ts, RH_s)
        q_air = specific_humidity(Ts, RH_air)
        LE = LE(u, q_s, q_air)

        return energy_balance(SWin, SWout, LWin, LWout, SH, LE)

    # Initial guess for Ts (e.g., ambient air temperature)
    initial_guess = Tair
    solution = root(equation, initial_guess)

    # Check if the solution converged
    if solution.success:
        return solution.x[0]
    else:
        raise ValueError("Solution did not converge")

# Assuming the solve_Ts function is already defined

def calculate_surface_temperatures(df):
    """
    Calculate surface temperatures using the time series data from a DataFrame.

    Parameters:
        df (DataFrame): DataFrame with columns: 'SWin', 'SWout', 'LWin', 'LWout', 'Tair', 'u', 'q_air', 'RHs'

    Returns:
        np.array: Array of calculated surface temperatures (Ts) for each row in the DataFrame.
    """
    Ts_values = []
    for _, row in df.iterrows():
        Ts = solve_Ts(
            SWin=row['SWin'], 
            SWout=row['SWout'], 
            LWin=row['LWin'], 
            LWout=row['LWout'], 
            Tair=row['Tair'], 
            u=row['u'], 
            RH_air=row['RH_air'], 
            prec =row['total_precipitation'], 
            snow_cover =row['snow_cover']
        )
        Ts_values.append(Ts)
    return np.array(Ts_values)

# Example: Calculate Ts for each time step
# Ts_values = calculate_surface_temperatures(SWin, SWout, LWin, LWout, Tair, u, q_air, RHs)